In [125]:
%load_ext autoreload
%autoreload 2

from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn


from model import *
import pickle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
#torch.set_grad_enabled(False)

In [50]:
# loading image data
noisy_imgs_1, noisy_imgs_2 = torch.load('../../data/train_data.pkl')
noisy_imgs, clean_imgs = torch.load('../../data/val_data.pkl')

train_input, train_target = noisy_imgs_1.float()/255.0, noisy_imgs_2.float()/255.0 
test_input, test_target = noisy_imgs.float()/255.0, clean_imgs.float()/255.0

In [23]:
# take batch of size 1
input_batch = train_input[:10,:,:,:]
target_batch = train_target[:10,:,:,:]
input_batch.shape

torch.Size([10, 3, 32, 32])

In [24]:
# create some tensors for simple tests
x = torch.randn((1, 3, 4, 4))

# y = torch.ones(x.shape)
y = torch.ones((1, 3, 4, 4))

### Testing our Conv2d Layer 

Forward pass

In [25]:
input_batch.shape, target_batch.shape

(torch.Size([10, 3, 32, 32]), torch.Size([10, 3, 32, 32]))

In [87]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2, stride=2)
output = my_conv.forward(input_batch)
output.shape

torch.Size([10, 4, 16, 16])

In [27]:
torch.testing.assert_allclose(my_conv.forward(input_batch), torch.nn.functional.conv2d(input_batch, my_conv.weight, my_conv.bias, stride=2))

Backward pass

In [47]:
model = nn.Conv2d(3, 4, 2, stride=2)
out = model(input_batch)
target_out = model(target_batch)
loss = nn.MSELoss()
l = loss(out, target_out)
l.requires_grad_()

l.backward()

In [45]:
print(model.weight.grad)

None


In [28]:
back = my_conv.backward(output)
back.shape

torch.Size([10, 3, 32, 32])

### Testing our Upsampling (TransposeConv2d) layer

Forward pass

In [29]:
input_batch.shape

torch.Size([10, 3, 32, 32])

In [76]:
my_t_conv = Upsampling(3, 5, 2, stride=2)
upsampled = my_t_conv.forward(input_batch)
upsampled.shape

torch.Size([10, 5, 64, 64])

In [31]:
torch.testing.assert_allclose(my_t_conv.forward(input_batch), torch.nn.functional.conv_transpose2d(input_batch, my_t_conv.weight, my_t_conv.bias, stride=2))

Backward pass

In [32]:
back_t = my_t_conv.backward(upsampled)
back_t.shape

torch.Size([10, 3, 32, 32])

### Sequential model testing

In [33]:
model = Sequential(Conv2d(3,4,2), ReLU())
forward_pass = model.forward(x)
backward_pass = model.backward(forward_pass)


### Testing model

In [120]:
train_input_batch = train_input[:1000, :, :, :]
train_target_batch = train_target[:1000, :, :, :]
train_input_batch.shape

torch.Size([1000, 3, 32, 32])

Train the model 

In [126]:
model = Model()
model.train(train_input_batch, train_target_batch, 50)

0 iteration: loss=0.7332425713539124
1 iteration: loss=0.7236798405647278
2 iteration: loss=0.7235281467437744
3 iteration: loss=0.7233770489692688
4 iteration: loss=0.7232725024223328
5 iteration: loss=0.7231711745262146
6 iteration: loss=0.7230492234230042
7 iteration: loss=0.7229362726211548
8 iteration: loss=0.722869336605072
9 iteration: loss=0.7227900624275208
10 iteration: loss=0.7227004170417786
11 iteration: loss=0.7226137518882751
12 iteration: loss=0.7225151062011719
13 iteration: loss=0.7224360704421997
14 iteration: loss=0.7223598957061768
15 iteration: loss=0.7222872972488403
16 iteration: loss=0.7222217917442322
17 iteration: loss=0.7221630811691284
18 iteration: loss=0.7221091985702515
19 iteration: loss=0.7220585942268372
20 iteration: loss=0.7220103740692139
21 iteration: loss=0.7219637632369995
22 iteration: loss=0.7219168543815613
23 iteration: loss=0.7218676209449768
24 iteration: loss=0.7218151688575745
25 iteration: loss=0.7217597365379333
26 iteration: loss=0.72

Prediction and psnr score

In [127]:
def psnr(denoised,ground_truth):
    mse=torch.mean((denoised-ground_truth)**2)
    return  - 10 * torch.log10(((denoised-ground_truth) ** 2).mean((1,2,3))).mean()

In [128]:
prediction = model.predict(test_input)
psnr(prediction, test_target)

tensor(12.7144)

Save the model

In [129]:
pickle.dump(model, open('bestmodel.pkl', 'wb'))